In [1]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00


In [2]:
#Import needed libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
import itertools

In [3]:
# Load dataset
df = pd.read_parquet('/content/drive/MyDrive/MIMIC-III_final_dataset_with_diagnoses.parquet')

In [4]:
df.head()

,ICUSTAY_ID,CHARTTIME,DiastolicBP,HeartRate,InspiredO2Fraction,O2Flow,RespiratoryRate,SpO2,SystolicBP,Temperature,...,aPTT,WEIGHT,HADM_ID,CHARTDATE,SPEC_TYPE_DESC,ORG_NAME,INTERPRETATION,ICUSTAY_ID_x,ICUSTAY_ID_y,ICD9_CODE
0,200001,2181-11-18 11:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,61.0,None,NaT,None,None,None,NaN,NaN,None
1,200001,2181-11-18 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,61.0,None,NaT,None,None,None,NaN,NaN,None
2,200001,2181-11-18 15:29:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.0,61.0,None,NaT,None,None,None,NaN,NaN,None
3,200001,2181-11-18 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.6,61.0,None,NaT,None,None,None,NaN,NaN,None
4,200001,2181-11-19 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.3,61.0,None,NaT,None,None,None,NaN,NaN,None


In [5]:
# Dimensions of the dataset
print("Dataset shape:", df.shape)

# Data types
print(df.info())

Dataset shape: (118980120, 29)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118980120 entries, 0 to 118980119
Data columns (total 29 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ICUSTAY_ID             object        
 1   CHARTTIME              datetime64[ns]
 2   DiastolicBP            float64       
 3   HeartRate              float64       
 4   InspiredO2Fraction     float64       
 5   O2Flow                 float64       
 6   RespiratoryRate        float64       
 7   SpO2                   float64       
 8   SystolicBP             float64       
 9   Temperature            float64       
 10  Altered_Mental_Status  float64       
 11  Oliguria               float64       
 12  CRP                    float64       
 13  INR                    float64       
 14  Lactate                float64       
 15  PCT                    float64       
 16  Platelets              float64       
 17  ScvO2                  float64

In [6]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 ICUSTAY_ID                       0
CHARTTIME                  1291626
DiastolicBP               73338734
HeartRate                 73338734
InspiredO2Fraction        73338734
O2Flow                    73338734
RespiratoryRate           73338734
SpO2                      73338734
SystolicBP                73338734
Temperature               73338734
Altered_Mental_Status     97583738
Oliguria                  73797444
CRP                      117198494
INR                      112528584
Lactate                  109516064
PCT                      117615988
Platelets                108904320
ScvO2                    116030026
WBC                      118969172
aPTT                     112118258
WEIGHT                    64542128
HADM_ID                          0
CHARTDATE                100775808
SPEC_TYPE_DESC           100775808
ORG_NAME                 100775808
INTERPRETATION           103737004
ICUSTAY_ID_x             100819208
ICUSTAY_ID_y             10

In [7]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
CHARTTIME,117688494,2152-12-02 22:25:44.214181888,2089-11-03 22:30:00,2128-12-15 15:00:00,2153-08-13 03:10:00,2177-10-02 21:40:00,2210-08-24 05:53:00,NaN
DiastolicBP,45641386.0,25.793596,-2.0,0.0,0.0,57.0,114108.98,246.986362
HeartRate,45641386.0,77.153435,-88.0,66.0,82.0,97.0,9999999.0,5538.688202
InspiredO2Fraction,45641386.0,8.345926,0.0,0.0,0.0,0.0,10050.0,21.996711
O2Flow,45641386.0,0.366498,-10.0,0.0,0.0,0.0,1210.0,2.266405
RespiratoryRate,45641386.0,17.570305,0.0,13.0,18.0,23.0,2355555.0,1304.633124
SpO2,45641386.0,82.727717,0.0,93.0,97.0,99.0,6363333.0,4015.639466
SystolicBP,45641386.0,47.688076,-69.0,0.0,0.0,110.0,141146.04,167.728969
Temperature,45641386.0,15.766592,-99.9,0.0,0.0,0.0,9637.0,36.571685
Altered_Mental_Status,21396382.0,0.318411,0.0,0.0,0.0,1.0,1.0,0.46586


In [8]:
# Check the type and content of the first non-null ICD9_CODE entries
sample_icd9 = df.loc[df['ICD9_CODE'].notnull(), 'ICD9_CODE'].head(10)
for idx, val in enumerate(sample_icd9):
    print(f"Row {idx}: type: {type(val)}, value: {val}")


Row 0: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 1: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 2: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 3: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 4: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 5: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 6: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 7: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
Row 8: type: <class 'numpy.ndarray'>, value: ['4280' '41401' '4240' '486' '42732' '51881' '5990' '5845' '496']
R

In [9]:
#Labeling Target variable
sepsis_icd9_codes = ['99591', '99592', '78552']

# Handle arrays correctly
df['Sepsis'] = df['ICD9_CODE'].apply(
    lambda codes: 1 if codes is not None and any(code in sepsis_icd9_codes for code in codes) else 0
)

# Verify the distribution of the target variable
print(df['Sepsis'].value_counts())


Sepsis
0    118430341
1       549779
Name: count, dtype: int64


In [10]:
# Handling Missing Values
missing_percentage = df.isnull().mean() * 100
print(missing_percentage[missing_percentage > 0])


CHARTTIME                 1.085581
DiastolicBP              61.639486
HeartRate                61.639486
InspiredO2Fraction       61.639486
O2Flow                   61.639486
RespiratoryRate          61.639486
SpO2                     61.639486
SystolicBP               61.639486
Temperature              61.639486
Altered_Mental_Status    82.016843
Oliguria                 62.025021
CRP                      98.502585
INR                      94.577635
Lactate                  92.045683
PCT                      98.853479
Platelets                91.531526
ScvO2                    97.520515
WBC                      99.990798
aPTT                     94.232766
WEIGHT                   54.246145
CHARTDATE                84.699703
SPEC_TYPE_DESC           84.699703
ORG_NAME                 84.699703
INTERPRETATION           87.188519
ICUSTAY_ID_x             84.736179
ICUSTAY_ID_y             84.736179
ICD9_CODE                98.855193
dtype: float64


In [11]:
# Select numeric columns only
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Fill missing numeric values with median
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


Building the model

In [12]:
df = df.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'])


In [13]:
features = ['DiastolicBP', 'HeartRate', 'InspiredO2Fraction',
            'O2Flow', 'RespiratoryRate', 'SpO2', 'SystolicBP',
            'Temperature', 'aPTT', 'WEIGHT']

target = 'Sepsis'


In [14]:
#Sort  data by patient and time:
df.sort_values(by=['ICUSTAY_ID', 'CHARTTIME'], inplace=True)

In [15]:
#Group data and create sequences:



sequence_length = 10  # adjust based on clinical logic


sequences = []
targets = []

# Group by patient ICUSTAY_ID
for patient_id, group in df.groupby('ICUSTAY_ID'):
    patient_features = group[features].values

    # Handle sequences shorter than the defined length by padding
    if len(patient_features) >= sequence_length:
        seq = patient_features[-sequence_length:]
    else:
        padding = np.zeros((sequence_length - len(group), len(features)))
        seq = np.vstack((padding, group[features].values))

    sequences.append(seq)

    # Define target as whether patient ever had Sepsis
    target = group['Sepsis'].max()
    targets.append(target)




In [16]:
# Convert to Numpy arrays and check shapes
X = np.array(sequences)
y = np.array(targets)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (83356, 10, 10)
Shape of y: (83356,)


In [17]:
# Check target distribution
print("Target variable distribution:", np.bincount(y))

Target variable distribution: [78382  4974]


In [18]:
# Split your data into training and testing sets (e.g., 80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X.shape}, {y.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")


Training set shape: (83356, 10, 10), (83356,)
Testing set shape: (16672, 10, 10), (16672,)


In [19]:
from sklearn.preprocessing import MinMaxScaler

# Create scaler instance
scaler = MinMaxScaler()

# Fit scaler on training data only, then apply to both train/test
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# Check result
print("Scaled Training data shape:", X_train_scaled.shape)
print("Scaled Testing data shape:", X_test_scaled.shape)


Scaled Training data shape: (66684, 10, 10)
Scaled Testing data shape: (16672, 10, 10)


In [20]:
# Define the LSTM model
model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [21]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [22]:
# Model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 10, 64)              │          19,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 32,161 (125.63 KB)

 Trainable params: 32,161 (125.63 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Training the Model
history = model.fit(
    X_train_scaled, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9399 - loss: 0.2528 - val_accuracy: 0.9402 - val_loss: 0.2265
Epoch 2/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9399 - loss: 0.2254 - val_accuracy: 0.9402 - val_loss: 0.2232
Epoch 3/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9407 - loss: 0.2213 - val_accuracy: 0.9402 - val_loss: 0.2198
Epoch 4/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9415 - loss: 0.2183 - val_accuracy: 0.9402 - val_loss: 0.2159
Epoch 5/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9412 - loss: 0.2156 - val_accuracy: 0.9402 - val_loss: 0.2102
Epoch 6/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9401 - loss: 0.2177 - val_accuracy: 0.9402 - val_loss: 0.2131
Epoch 7/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9391 - loss: 0.2217 - val_accuracy: 0.9402 - val_loss: 0.2124
Epoch 8/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9398 - loss: 0

In [26]:
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2%}")

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9419 - loss: 0.2129
Test Accuracy: 94.03%


In [27]:
# Predicting and checking other metrics

y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [28]:
# Classification report
classification_report(y_test, y_pred)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


'              precision    recall  f1-score   support\n\n           0       0.94      1.00      0.97     15677\n           1       0.00      0.00      0.00       995\n\n    accuracy                           0.94     16672\n   macro avg       0.47      0.50      0.48     16672\nweighted avg       0.88      0.94      0.91     16672\n'

In [29]:
# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[15677     0]
 [  995     0]]


In [30]:
# ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC Score: {roc_auc:.2%}")

ROC-AUC Score: 49.95%


In [31]:
def create_gru_model(input_shape):
    model = Sequential([
        GRU(64, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        GRU(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(0.001),
                  metrics=['accuracy'])
    return model

In [32]:
# Create the GRU model instance
gru_model = create_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [33]:
# Check model summary
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 10, 64)              │          14,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 32)                  │           9,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,545 (95.88 KB)

 Trainable params: 24,545 (95.88 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
history_gru = gru_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

Epoch 1/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.9084 - loss: 0.4518 - val_accuracy: 0.9402 - val_loss: 0.2254
Epoch 2/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9399 - loss: 0.2252 - val_accuracy: 0.9402 - val_loss: 0.2130
Epoch 3/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9420 - loss: 0.2123 - val_accuracy: 0.9402 - val_loss: 0.2264
Epoch 4/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9398 - loss: 0.2182 - val_accuracy: 0.9402 - val_loss: 0.2123
Epoch 5/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9389 - loss: 0.2155 - val_accuracy: 0.9402 - val_loss: 0.2167
Epoch 6/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9413 - loss: 0.2116 - val_accuracy: 0.9402 - val_loss: 0.2149
Epoch 7/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9412 - loss: 0.2097 - val_accuracy: 0.9402 - val_loss: 0.2143
Epoch 8/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9405 - loss

In [35]:
# Evaluate on test data
gru_loss, gru_accuracy = gru_model.evaluate(X_test, y_test)
print(f"GRU Test Accuracy: {gru_accuracy:.2%}")

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9419 - loss: 0.2011
GRU Test Accuracy: 94.03%


In [36]:
# Generate predictions
y_gru_pred_proba = gru_model.predict(X_test).flatten()
y_gru_pred = (y_gru_pred_proba > 0.5).astype(int)

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


In [37]:
# Classification report
print("\nClassification Report:\n", classification_report(y_test, y_gru_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97     15677
           1       0.00      0.00      0.00       995

    accuracy                           0.94     16672
   macro avg       0.47      0.50      0.48     16672
weighted avg       0.88      0.94      0.91     16672



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
# Confusion matrix
confusion_matrix(y_test, y_gru_pred)

array([[15677,     0],
       [  995,     0]])

In [39]:
# ROC-AUC score
gru_roc_auc = roc_auc_score(y_test, y_gru_pred_proba)
print(f"GRU ROC-AUC Score: {gru_roc_auc:.2%}")

GRU ROC-AUC Score: 66.27%


In [40]:
!pip install scikeras

In [41]:
def create_lstm(units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        LSTM(units, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        LSTM(units//2, activation='relu'),
        Dropout(dropout_rate),
        Dense(units//4, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [42]:
units_list = [32, 64]
dropout_rates = [0.2, 0.3]
learning_rates = [0.001, 0.0001]

best_auc = 0
best_params = {}

for units, dropout_rate, learning_rate in itertools.product(units_list, dropout_rates, learning_rates):
    print(f"\nTesting combination: units={units}, dropout={dropout_rate}, lr={learning_rate}")

    model = create_lstm(units, dropout_rate, learning_rate, input_shape=(X_train.shape[1], X_train.shape[2]))
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=0)

    # Evaluate on test set
    y_pred_proba = model.predict(X_test).flatten()
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC-AUC Score: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_params = {
            'units': units,
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate
        }

print("\n  Best Hyperparameters found:")
print(best_params)
print(f"Best ROC-AUC Score: {best_auc:.4f}")


Testing combination: units=32, dropout=0.2, lr=0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC Score: 0.6194

Testing combination: units=32, dropout=0.2, lr=0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC Score: 0.6803

Testing combination: units=32, dropout=0.3, lr=0.001


...

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC Score: 0.6857

Testing combination: units=64, dropout=0.3, lr=0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
ROC-AUC Score: 0.6862

Testing combination: units=64, dropout=0.3, lr=0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC Score: 0.6730

  Best Hyperparameters found:
{'units': 64, 'dropout_rate': 0.3, 'learning_rate': 0.001}
Best ROC-AUC Score: 0.6862


In [43]:
final_model = create_lstm(
    best_params['units'],
    best_params['dropout_rate'],
    best_params['learning_rate'],
    input_shape=(X_train.shape[1], X_train.shape[2])
)

# Train final model
final_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate final model
y_final_pred = (final_model.predict(X_test) > 0.5).astype(int)
y_final_pred_proba = final_model.predict(X_test).flatten()

print("\n Final Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_final_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_final_pred_proba):.4f}")


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1876/1876 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.9176 - loss: 0.4867 - val_accuracy: 0.9402 - val_loss: 0.2328
Epoch 2/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9406 - loss: 0.2267 - val_accuracy: 0.9402 - val_loss: 0.2230
Epoch 3/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9412 - loss: 0.2202 - val_accuracy: 0.9402 - val_loss: 0.2245
Epoch 4/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9392 - loss: 0.2271 - val_accuracy: 0.9402 - val_loss: 0.2231
Epoch 5/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9390 - loss: 0.2365 - val_accuracy: 0.9402 - val_loss: 0.2242
Epoch 6/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9400 - loss: 0.2237 - val_accuracy: 0.9402 - val_loss: 0.2219
Epoch 7/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9401 - loss: 0.2229 - val_accuracy: 0.9402 - val_loss: 0.2199
Epoch 8/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9417 - loss: 0.2180 - v

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import itertools
from sklearn.metrics import roc_auc_score

def build_gru_model(units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        GRU(units, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        GRU(units // 2, activation='relu'),
        Dropout(dropout_rate),
        Dense(units // 4, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [45]:
units_list = [32, 64]
dropout_rates = [0.2, 0.3]
learning_rates = [0.001, 0.0001]

best_auc = 0
best_params = {}

for units, dropout_rate, learning_rate in itertools.product(units_list, dropout_rates, learning_rates):
    print(f"\nTraining GRU: units={units}, dropout={dropout_rate}, lr={learning_rate}")

    # Create model
    model = build_gru_model(input_shape=(X_train.shape[1], X_train.shape[2]),
                            units=units,
                            dropout_rate=dropout_rate,
                            learning_rate=learning_rate)

    # Train the model briefly
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=0)

    # Evaluate on test set
    y_pred_proba = model.predict(X_test).flatten()
    auc = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC-AUC: {auc:.4f}")

    # Update best params if better
    if auc > best_auc:
        best_auc = auc
        best_params = {'units': units, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate}

print("\n Best Hyperparameters for GRU:")
print(best_params)
print(f"Best ROC-AUC Score: {best_auc:.4f}")



Training GRU: units=32, dropout=0.2, lr=0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC: 0.6813

Training GRU: units=32, dropout=0.2, lr=0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC: 0.6581

Training GRU: units=32, dropout=0.3, lr=0.001


...

521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC: 0.6861

Training GRU: units=64, dropout=0.3, lr=0.001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC: 0.6248

Training GRU: units=64, dropout=0.3, lr=0.0001


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


521/521 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
ROC-AUC: 0.6845

 Best Hyperparameters for GRU:
{'units': 64, 'dropout_rate': 0.2, 'learning_rate': 0.0001}
Best ROC-AUC Score: 0.6861


In [46]:
# Train final GRU model with best parameters
final_gru_model = build_gru_model(
    units=best_params['units'],
    dropout_rate=best_params['dropout_rate'],
    learning_rate=best_params['learning_rate'],
    input_shape=(X_train.shape[1], X_train.shape[2])
)

# Fit the model
final_gru_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate
y_pred_proba_final = final_gru_model.predict(X_test).flatten()
y_pred_final = (y_pred_proba_final > 0.5).astype(int)

final_accuracy = accuracy_score(y_test, y_pred_final)
final_auc = roc_auc_score(y_test, y_pred_proba_final)

print("\n Final GRU Model Performance:")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"ROC-AUC: {final_auc:.4f}")


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1876/1876 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.9125 - loss: 0.2938 - val_accuracy: 0.9402 - val_loss: 0.2330
Epoch 2/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9407 - loss: 0.2128 - val_accuracy: 0.9402 - val_loss: 0.2211
Epoch 3/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9403 - loss: 0.2076 - val_accuracy: 0.9402 - val_loss: 0.2170
Epoch 4/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9412 - loss: 0.2032 - val_accuracy: 0.9402 - val_loss: 0.2244
Epoch 5/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9422 - loss: 0.2007 - val_accuracy: 0.9402 - val_loss: 0.2126
Epoch 6/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9410 - loss: 0.2021 - val_accuracy: 0.9402 - val_loss: 0.2163
Epoch 7/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9409 - loss: 0.2022 - val_accuracy: 0.9402 - val_loss: 0.2101
Epoch 8/20
1876/1876 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9392 - loss: 0.205